In [1]:
from datasets import load_dataset

squad = load_dataset("squad", split="train")
squad = squad.train_test_split(test_size=0.2, seed=42)

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [70]:
import pandas as pd

In [103]:
import pandas as pd
from datasets import load_from_disk, load_dataset, concatenate_datasets, Dataset

# 1️⃣ Load existing dataset
dataset_fr = load_from_disk("squad_backtranslated/")

# 2️⃣ Load CSV
# This returns a DatasetDict, usually with a 'train' split
dataset_de = load_dataset('csv', data_files="augmented_squad.csv")



In [104]:
import re
from datasets import Dataset

def parse_answers_from_string(example):
    """Parse string representation of dict with numpy arrays"""
    answers_str = example['answers']
    
    # Extract text using regex
    text_match = re.search(r"'text': array\(\['(.+?)'\]", answers_str)
    text_value = text_match.group(1) if text_match else ""
    
    # Extract answer_start using regex
    start_match = re.search(r"'answer_start': array\(\[(\d+)\]", answers_str)
    start_value = int(start_match.group(1)) if start_match else 0
    
    # Convert to HF format (lists)
    example['answers'] = {
        'text': [text_value],
        'answer_start': [start_value]
    }
    
    return example

# Apply the conversion
dataset_de_converted = dataset_de['train'].map(parse_answers_from_string)

# Verify
print("Before:")
print(dataset_de['train']['answers'][1])
print("\nAfter:")
print(dataset_de_converted['answers'][1])
print("\nFrench (target):")
print(dataset_fr['train']['answers'][1])

# Cast to match exact features
dataset_de_converted = dataset_de_converted.cast(dataset_fr['train'].features)

# Concatenate
from datasets import concatenate_datasets
combined_dataset = concatenate_datasets([dataset_fr['train'], dataset_de_converted])
print(f"\nSuccess! Combined dataset size: {len(combined_dataset)}")

Map:   0%|          | 0/70079 [00:00<?, ? examples/s]

Before:
{'answer_start': array([316]), 'text': array(['US$10,000'], dtype=object)}

After:
{'answer_start': [316], 'text': ['US$10,000']}

French (target):
{'text': ['US$10,000'], 'answer_start': [316]}


Casting the dataset:   0%|          | 0/70079 [00:00<?, ? examples/s]


Success! Combined dataset size: 140158


In [118]:
combined_dataset


Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 140158
})

In [116]:
import copy
augmented_squad = copy.deepcopy(squad)
augmented_squad['train'] = combined_dataset


In [117]:
augmented_squad

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 140158
    })
    test: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 17520
    })
})

In [123]:
print(squad['train']['question'][5])
print(augmented_squad['train']['question'][5])
print(augmented_squad['train']['question'][70084])

What is the nearest port to Saint Helena?
Which port is closest to St. Helena?
What is the nearest port to St. Helena?


In [134]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="longest_first",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [132]:
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()

In [135]:
tokenized_squad = augmented_squad.map(
    preprocess_function,
    batched=True,
    remove_columns=augmented_squad["train"].column_names,
)

Map:   0%|          | 0/140158 [00:00<?, ? examples/s]

Map:   0%|          | 0/17520 [00:00<?, ? examples/s]

In [137]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained("./drill04_qa_model")

# Freeze first 2 transformer layers
for layer in model.distilbert.transformer.layer[:2]:
    for param in layer.parameters():
        param.requires_grad = False

# Check which parameters are trainable
trainable_params = [n for n, p in model.named_parameters() if p.requires_grad]
print(f"Trainable parameters: {len(trainable_params)}")


Trainable parameters: 70


In [140]:
# ✅ Use GPU if available
import torch
from transformers import Trainer, TrainingArguments

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"There are {torch.cuda.device_count()} GPU(s) available.")
    print("Device name:", torch.cuda.get_device_name(0))
    model.to(device)
else:
    device = torch.device("cpu")
    print("No GPU found, using CPU.")

# ✅ Training configuration (optimized for disk usage)
training_args = TrainingArguments(
    output_dir="drill04+DE_Backtranslated",       # folder to save model
    eval_strategy="epoch",         # correct parameter name
    learning_rate=5e-6,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy="epoch",               # save checkpoint only once per epoch
    save_total_limit=1,                  # keep only the last checkpoint
    load_best_model_at_end=True,         # optional, keeps best checkpoint
    push_to_hub=False,
    report_to="none",                    # disable wandb or tensorboard logs
    logging_dir=None,                    # avoid creating logging folders
    #fp16=True
)

# ✅ Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,                 # fixed from "processing_class"
    data_collator=data_collator,
)

trainer.train()


There are 1 GPU(s) available.
Device name: NVIDIA GeForce RTX 3050 Laptop GPU


C:\Users\Jettanat\AppData\Local\Temp\ipykernel_24640\999138565.py:33: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.847400,1.125801
2,0.773400,1.141264


TrainOutput(global_step=17520, training_loss=0.8348809875854074, metrics={'train_runtime': 10254.8273, 'train_samples_per_second': 27.335, 'train_steps_per_second': 1.708, 'total_flos': 2.7468115791427584e+16, 'train_loss': 0.8348809875854074, 'epoch': 2.0})

In [141]:
# Save the model and tokenizer
trainer.save_model("./drill04+DE+FR_Backtranslated")
tokenizer.save_pretrained("./drill04+DE+FR_Backtranslated")

('./drill04+DE+FR_Backtranslated\\tokenizer_config.json',
 './drill04+DE+FR_Backtranslated\\special_tokens_map.json',
 './drill04+DE+FR_Backtranslated\\vocab.txt',
 './drill04+DE+FR_Backtranslated\\added_tokens.json',
 './drill04+DE+FR_Backtranslated\\tokenizer.json')

In [142]:
max_length = 384
stride = 128


def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

validation_dataset = squad["test"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=squad["test"].column_names,
)
len(squad["test"]), len(validation_dataset)

Map:   0%|          | 0/17520 [00:00<?, ? examples/s]

(17520, 17688)

In [143]:
from tqdm.auto import tqdm
import collections
import numpy as np
import evaluate
metric = evaluate.load("squad")

n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits, features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        # Loop through all features associated with that example
        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    # Skip answers with a length that is either < 0 or > max_answer_length
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],
                    }
                    answers.append(answer)

        # Select the answer with the best score
        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append(
                {"id": example_id, "prediction_text": best_answer["text"]}
            )
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})

    theoretical_answers = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return metric.compute(predictions=predicted_answers, references=theoretical_answers)

In [144]:
predictions, _, _ = trainer.predict(validation_dataset)
start_logits, end_logits = predictions
compute_metrics(start_logits, end_logits, validation_dataset, squad["test"])

  0%|          | 0/17520 [00:00<?, ?it/s]

{'exact_match': 64.46917808219177, 'f1': 78.29335437423069}

In [146]:
question = "Which is blue's car price"
context="The blue car costs $20,000 and the red car costs $25,000."
print(question)
from transformers import pipeline
question_answerer = pipeline("question-answering",
                             model="./drill04_qa_model",
                            tokenizer="./drill04_qa_model",
                            fp16=True)

result = question_answerer(question=question, context=context)
print(result['answer'])

Device set to use cuda:0


Which is blue's car price
$20,000 and the red car costs $25,000
